In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
import pandas as pd
trn_set = pd.read_csv(io.BytesIO(uploaded['trn.csv']))

In [ ]:
def load_data(dataframe):
    texts = dataframe["tweet"].tolist()
    labels = dataframe["type"].tolist()
    return texts, labels

In [ ]:
# Read all the documents from the directory

texts, labels = load_data(trn_set)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
     
## =============================================
## TODO: Add your implementation here
vectorizer = CountVectorizer(lowercase=True, min_df=5, ngram_range=(1,1), tokenizer=lambda x : x.split(), binary=True)
BoW = vectorizer.fit_transform(texts)
print("Dimensionality:", BoW.shape)

Dimensionality: (111291, 19653)


In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from numpy.linalg import norm
from sklearn.preprocessing import normalize

K = 2 # number of clusters

In [ ]:
## =============================================
## TODO: Add your implementation here
text_normalized = normalize(BoW)
kmeans = KMeans(n_clusters=K, random_state=1)
kmeans.fit(text_normalized)
indices_bow = kmeans.labels_

In [ ]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

## =============================================
## TODO: Add your implementation here
svd = TruncatedSVD(n_components=200, n_iter=5)
embeddings = svd.fit_transform(BoW)
embeddings_normalized = normalize(embeddings)
embeddings_normalized.shape

(111291, 200)

In [ ]:
## =============================================
## TODO: Add your implementation here
kmeans_dense = KMeans(n_clusters=K, random_state=1)
kmeans_dense.fit(embeddings_normalized)
indices_embeddings = kmeans_dense.labels_

In [ ]:
import numpy as np

def eval(labels, cluster_indices):
    """
    labels: ground-truth labels of all examples
    cluster_indices: cluster indices of all examples
    """
    ## =============================================
    ## TODO: Add your implementation here
    clusters = {}
    for index, label in enumerate(cluster_indices):
      clusters.setdefault(label, []).append(index)
    acc = 0
    for i in range(len(clusters)):
      indices = clusters[i]
      counts = {}
      for index in indices:
        counts.setdefault(labels[index], 0)
        counts[labels[index]] += 1
      maxCategory = 0
      for count in counts:
        maxCategory = max(maxCategory, counts[count])
      acc += maxCategory
    purity = acc / len(labels)
    
    return purity

In [ ]:
print("The purity score of clustering results with BoW representations:")
print(eval(labels, indices_bow))
print("The purity score of clustering results with low-dimensional representations: ")
print(eval(labels, indices_embeddings))

The purity score of clustering results with BoW representations:
0.8693605053418515
The purity score of clustering results with low-dimensional representations: 
0.8691089126703866
